In [1]:
#! pip install gensim

In [2]:
#! pip install nltk

In [3]:
#! pip install stop-words

In [4]:
#! pip install pyLDAvis

In [5]:
import os 
import shutil
import pandas as pd
import numpy as np 
import gensim  
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
pyLDAvis.enable_notebook()
import nltk 
nltk.download('wordnet')
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *  

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Miguel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
df=pd.read_csv('recipes.csv')

In [7]:
df.head(5)

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


In [8]:
df.drop(labels=['Unnamed: 0','Ingredients'], axis=1, inplace=True)

In [9]:
df.head(5)

,Title,Instructions,Image_Name,Cleaned_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,Crispy Salt and Pepper Potatoes,Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,Thanksgiving Mac and Cheese,Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,Italian Sausage and Bread Stuffing,Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,Newton's Law,Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13501 entries, 0 to 13500
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Title                13496 non-null  object
 1   Instructions         13493 non-null  object
 2   Image_Name           13501 non-null  object
 3   Cleaned_Ingredients  13501 non-null  object
dtypes: object(4)
memory usage: 422.0+ KB


In [11]:
df['Cleaned_Ingredients'][0]

'[\'1 (3½–4-lb.) whole chicken\', \'2¾ tsp. kosher salt, divided, plus more\', \'2 small acorn squash (about 3 lb. total)\', \'2 Tbsp. finely chopped sage\', \'1 Tbsp. finely chopped rosemary\', \'6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature\', \'¼ tsp. ground allspice\', \'Pinch of crushed red pepper flakes\', \'Freshly ground black pepper\', \'⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)\', \'2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces\', \'2 Tbsp. extra-virgin olive oil\', \'½ small red onion, thinly sliced\', \'3 Tbsp. apple cider vinegar\', \'1 Tbsp. white miso\', \'¼ cup all-purpose flour\', \'2 Tbsp. unsalted butter, room temperature\', \'¼ cup dry white wine\', \'2 cups unsalted chicken broth\', \'2 tsp. white miso\', \'Kosher salt\', \'freshly ground pepper\']'

In [12]:
stemmer = SnowballStemmer('english')

In [13]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [14]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Miguel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [15]:
with np.load('recipes2.npz', allow_pickle=True) as f:
    ingredients = f['ingredients']

In [16]:
ingredients.shape

(3500,)

In [17]:
processed = df['Cleaned_Ingredients'].map(preprocess)

In [18]:
processed 

0        [chicken, kosher, salt, divid, plus, small, ac...
1        [larg, white, pound, potato, inch, diamet, tea...
2        [evapor, milk, milk, garlic, powder, onion, po...
3        [pound, round, italian, loaf, inch, cub, cup, ...
4        [teaspoon, dark, brown, sugar, teaspoon, water...
                               ...                        
13496    [purpos, flour, unsweeten, cocoa, powder, teas...
13497    [preserv, lemon, pound, butternut, squash, pee...
13498    [leftov, katsuo, bushi, dri, bonito, flake, ma...
13499    [stick, plus, tablespoon, unsalt, butter, babi...
13500    [medium, larg, fresh, poblano, chile, tomato, ...
Name: Cleaned_Ingredients, Length: 13501, dtype: object

In [19]:
from stop_words import get_stop_words
stop_words = get_stop_words('english')

In [20]:
processed2 = processed.apply(lambda x: list(set(x) & set(ingredients)))

In [21]:
df['words'] = processed2

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13501 entries, 0 to 13500
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Title                13496 non-null  object
 1   Instructions         13493 non-null  object
 2   Image_Name           13501 non-null  object
 3   Cleaned_Ingredients  13501 non-null  object
 4   words                13501 non-null  object
dtypes: object(5)
memory usage: 527.5+ KB


In [23]:
df['words'].str.len() != 0

0        True
1        True
2        True
3        True
4        True
         ... 
13496    True
13497    True
13498    True
13499    True
13500    True
Name: words, Length: 13501, dtype: bool

In [24]:
df2 = df.loc[df['words'].str.len() != 0]

In [25]:
df2.reset_index(drop=True, inplace=True)

In [26]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13477 entries, 0 to 13476
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Title                13477 non-null  object
 1   Instructions         13475 non-null  object
 2   Image_Name           13477 non-null  object
 3   Cleaned_Ingredients  13477 non-null  object
 4   words                13477 non-null  object
dtypes: object(5)
memory usage: 526.6+ KB


In [27]:
processed3 = df2['words']

In [28]:
processed3

0        [broth, squash, wine, salt, flour, cider, blac...
1        [white, salt, parsley, thyme, pepper, black, p...
2        [macaroni, fresh, smoke, cream, extra, milk, s...
3        [broth, cream, chicken, rib, egg, parsley, but...
4        [fresh, bourbon, cinnamon, water, lemon, brown...
                               ...                        
13472    [cream, egg, vanilla, milk, salt, water, flour...
13473    [nut, fresh, peppercorn, squash, cinnamon, cou...
13474    [fresh, rice, dashi, water, bonito, toast, sug...
13475       [fresh, nutmeg, spinach, butter, phyllo, plus]
13476    [nut, salt, cilantro, garlic, black, cream, so...
Name: words, Length: 13477, dtype: object

In [29]:
filters = ['tablespoon', 'teaspoon', 'extract', 'extra', 'fresh', 'plus', 'kosher', 'instant', 'espresso', 'white', 'brown', 'black', 'pink', 'orange', 'green', 'blue', 'yellow', 'red']

In [30]:
processed3 = processed3.apply(lambda x: list(set(x) - set(filters)))

In [31]:
df2['words'] = processed3

C:\Users\Miguel\AppData\Local\Temp/ipykernel_1824/1055242072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['words'] = processed3


In [32]:
dictionary = gensim.corpora.Dictionary(processed3)

In [33]:
bow_corpus = [dictionary.doc2bow(x) for x in processed3]
bow_corpus[4310]

[(2, 1), (5, 1), (10, 1), (23, 1), (36, 1), (48, 1), (128, 1), (159, 1)]

In [34]:
bow_corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1)],
 [(8, 1), (10, 1), (14, 1), (15, 1), (16, 1)],
 [(7, 1),
  (8, 1),
  (10, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1)],
 [(1, 1),
  (2, 1),
  (3, 1),
  (7, 1),
  (14, 1),
  (18, 1),
  (19, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1)],
 [(2, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)],
 [(35, 1), (38, 1), (39, 1)],
 [(2, 1), (4, 1), (10, 1), (34, 1), (35, 1), (40, 1)],
 [(32, 1), (34, 1), (35, 1), (36, 1), (41, 1), (42, 1)],
 [(7, 1),
  (8, 1),
  (10, 1),
  (19, 1),
  (25, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1)],
 [(2, 1),
  (7, 1),
  (8, 1),
  (10, 1),
  (14, 1),
  (26, 1),
  (47, 1),
  (57, 1),
  (58, 1),
  (59, 1)],
 [(8, 1),
  (1

In [35]:
bow_doc_4310 = bow_corpus[4310] 
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 2 ("butter") appears 1 time.
Word 5 ("flour") appears 1 time.
Word 10 ("salt") appears 1 time.
Word 23 ("powder") appears 1 time.
Word 36 ("sugar") appears 1 time.
Word 48 ("ginger") appears 1 time.
Word 128 ("vanilla") appears 1 time.
Word 159 ("yolk") appears 1 time.


In [36]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

In [37]:
corpus_tfidf = tfidf[bow_corpus]

In [38]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.2769507991192922),
 (1, 0.2634054631531712),
 (2, 0.11659252522242741),
 (3, 0.21042922828265373),
 (4, 0.3449576253450679),
 (5, 0.1607893733968412),
 (6, 0.4809709623396407),
 (7, 0.148454511148991),
 (8, 0.0952599503872838),
 (9, 0.38991398102544755),
 (10, 0.040970463048483785),
 (11, 0.3999898770896642),
 (12, 0.18250832778160064),
 (13, 0.20428804259246577)]


In [39]:
len(bow_corpus[30])

6

In [40]:
#from gensim.models.coherencemodel import CoherenceModel
    
#for i in range(1,31):
    #lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=i, id2word=dictionary, workers=None)
    #cm = CoherenceModel(model=lda_model, corpus=bow_corpus, coherence='u_mass')
    #print(i,"topics:", cm.get_coherence())
    
#13 topics

In [41]:
lda_model_opt = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, workers=2, random_state=15)

In [42]:
for idx, topic in lda_model_opt.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.061*"salt" + 0.052*"garlic" + 0.050*"pepper" + 0.045*"clove" + 0.040*"onion" + 0.025*"butter" + 0.024*"tomato" + 0.023*"water" + 0.022*"parsley" + 0.019*"potato"
Topic: 1 
Words: 0.058*"sugar" + 0.049*"butter" + 0.049*"salt" + 0.030*"water" + 0.026*"flour" + 0.023*"cream" + 0.022*"clove" + 0.022*"lemon" + 0.020*"garlic" + 0.019*"cinnamon"
Topic: 2 
Words: 0.078*"salt" + 0.055*"sugar" + 0.044*"butter" + 0.043*"flour" + 0.033*"pepper" + 0.033*"powder" + 0.033*"cream" + 0.030*"egg" + 0.029*"milk" + 0.028*"lemon"
Topic: 3 
Words: 0.073*"salt" + 0.051*"pepper" + 0.035*"butter" + 0.032*"onion" + 0.031*"garlic" + 0.028*"vinegar" + 0.027*"wine" + 0.026*"lemon" + 0.024*"sugar" + 0.019*"tomato"
Topic: 4 
Words: 0.047*"sugar" + 0.046*"garlic" + 0.044*"salt" + 0.037*"clove" + 0.030*"lime" + 0.027*"pepper" + 0.025*"water" + 0.025*"ginger" + 0.022*"chile" + 0.019*"onion"


In [43]:
processed3[4310]

['ginger', 'vanilla', 'salt', 'flour', 'butter', 'sugar', 'yolk', 'powder']

In [44]:
for index, score in sorted(lda_model_opt[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_opt.print_topic(index, 10)))


Score: 0.908279538154602	 
Topic: 0.078*"salt" + 0.055*"sugar" + 0.044*"butter" + 0.043*"flour" + 0.033*"pepper" + 0.033*"powder" + 0.033*"cream" + 0.030*"egg" + 0.029*"milk" + 0.028*"lemon"

Score: 0.02324897237122059	 
Topic: 0.058*"sugar" + 0.049*"butter" + 0.049*"salt" + 0.030*"water" + 0.026*"flour" + 0.023*"cream" + 0.022*"clove" + 0.022*"lemon" + 0.020*"garlic" + 0.019*"cinnamon"

Score: 0.02322200872004032	 
Topic: 0.047*"sugar" + 0.046*"garlic" + 0.044*"salt" + 0.037*"clove" + 0.030*"lime" + 0.027*"pepper" + 0.025*"water" + 0.025*"ginger" + 0.022*"chile" + 0.019*"onion"

Score: 0.022641455754637718	 
Topic: 0.061*"salt" + 0.052*"garlic" + 0.050*"pepper" + 0.045*"clove" + 0.040*"onion" + 0.025*"butter" + 0.024*"tomato" + 0.023*"water" + 0.022*"parsley" + 0.019*"potato"

Score: 0.022608017548918724	 
Topic: 0.073*"salt" + 0.051*"pepper" + 0.035*"butter" + 0.032*"onion" + 0.031*"garlic" + 0.028*"vinegar" + 0.027*"wine" + 0.026*"lemon" + 0.024*"sugar" + 0.019*"tomato"


In [45]:
vis_data = gensimvis.prepare(lda_model_opt, bow_corpus, dictionary)

C:\Users\Miguel\anaconda3\envs\nova_ims\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [46]:
pyLDAvis.display(vis_data)

In [47]:
def format_topics_sentences(ldamodel=lda_model_opt, corpus=bow_corpus, texts=df2['words']):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model_opt, corpus=bow_corpus, texts=df2['words'])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,3.0,0.8326,"salt, pepper, butter, onion, garlic, vinegar, ...","[broth, squash, wine, sage, salt, bread, flour..."
1,1,0.0,0.8634,"salt, garlic, pepper, clove, onion, butter, to...","[salt, parsley, thyme, pepper, potato]"
2,2,2.0,0.5655,"salt, sugar, butter, flour, pepper, powder, cr...","[macaroni, smoke, cream, milk, salt, cheddar, ..."
3,3,0.0,0.9453,"salt, garlic, pepper, clove, onion, butter, to...","[broth, cream, rib, egg, onion, parsley, butte..."
4,4,1.0,0.8983,"sugar, butter, salt, water, flour, cream, clov...","[bourbon, cinnamon, water, lemon, garnish, but..."
5,5,1.0,0.7956,"sugar, butter, salt, water, flour, cream, clov...","[lemon, reposado, tequila]"
6,6,1.0,0.8833,"sugar, butter, salt, water, flour, cream, clov...","[peppercorn, salt, lemon, garnish, cider, butter]"
7,7,1.0,0.8840,"sugar, butter, salt, water, flour, cream, clov...","[bourbon, scotch, lemon, syrup, garnish, sugar]"
8,8,0.0,0.5080,"salt, garlic, pepper, clove, onion, butter, to...","[masala, salt, cilantro, garlic, grain, rice, ..."
9,9,3.0,0.9252,"salt, pepper, butter, onion, garlic, vinegar, ...","[ghee, soup, salt, parsley, cumin, butter, pep..."


In [48]:
df_dominant_topic.head(5)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,3.0,0.8326,"salt, pepper, butter, onion, garlic, vinegar, ...","[broth, squash, wine, sage, salt, bread, flour..."
1,1,0.0,0.8634,"salt, garlic, pepper, clove, onion, butter, to...","[salt, parsley, thyme, pepper, potato]"
2,2,2.0,0.5655,"salt, sugar, butter, flour, pepper, powder, cr...","[macaroni, smoke, cream, milk, salt, cheddar, ..."
3,3,0.0,0.9453,"salt, garlic, pepper, clove, onion, butter, to...","[broth, cream, rib, egg, onion, parsley, butte..."
4,4,1.0,0.8983,"sugar, butter, salt, water, flour, cream, clov...","[bourbon, cinnamon, water, lemon, garnish, but..."


In [49]:
df_labels=df_dominant_topic.iloc[:,1:4]

In [50]:
final_df = pd.concat([df2, df_labels], axis=1)

#final_df.drop(columns='labels',inplace=True)
final_df.rename(columns={'Dominant_Topic':'labels'}, inplace=True)

In [51]:
final_df

,Title,Instructions,Image_Name,Cleaned_Ingredients,words,labels,Topic_Perc_Contrib,Keywords
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","[broth, squash, wine, sage, salt, bread, flour...",3.0,0.8326,"salt, pepper, butter, onion, garlic, vinegar, ..."
1,Crispy Salt and Pepper Potatoes,Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (...","[salt, parsley, thyme, pepper, potato]",0.0,0.8634,"salt, garlic, pepper, clove, onion, butter, to..."
2,Thanksgiving Mac and Cheese,Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ...","[macaroni, smoke, cream, milk, salt, cheddar, ...",2.0,0.5655,"salt, sugar, butter, flour, pepper, powder, cr..."
3,Italian Sausage and Bread Stuffing,Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in...","[broth, cream, rib, egg, onion, parsley, butte...",0.0,0.9453,"salt, garlic, pepper, clove, onion, butter, to..."
4,Newton's Law,Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho...","[bourbon, cinnamon, water, lemon, garnish, but...",1.0,0.8983,"sugar, butter, salt, water, flour, cream, clov..."
...,...,...,...,...,...,...,...,...
13472,Brownie Pudding Cake,Preheat the oven to 350°F. Into a bowl sift to...,brownie-pudding-cake-14408,"['1 cup all-purpose flour', '2/3 cup unsweeten...","[cream, egg, vanilla, milk, water, salt, flour...",2.0,0.9375,"salt, sugar, butter, flour, pepper, powder, cr..."
13473,Israeli Couscous with Roasted Butternut Squash...,Preheat oven to 475°F.\nHalve lemons and scoop...,israeli-couscous-with-roasted-butternut-squash...,"['1 preserved lemon', '1 1/2 pound butternut s...","[nut, peppercorn, squash, cinnamon, couscous, ...",0.0,0.9372,"salt, garlic, pepper, clove, onion, butter, to..."
13474,Rice with Soy-Glazed Bonito Flakes and Sesame ...,"If using katsuo bushi flakes from package, moi...",rice-with-soy-glazed-bonito-flakes-and-sesame-...,['Leftover katsuo bushi (dried bonito flakes) ...,"[rice, dashi, water, bonito, toast, sugar, gra...",4.0,0.7928,"sugar, garlic, salt, clove, lime, pepper, wate..."
13475,Spanakopita,Melt 1 tablespoon butter in a 12-inch heavy sk...,spanakopita-107344,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,"[phyllo, nutmeg, spinach, butter]",1.0,0.8366,"sugar, butter, salt, water, flour, cream, clov..."


In [52]:
final_df['labels']=final_df['labels'].apply(lambda x:x+1) 
final_df['labels']=final_df['labels'].astype(int)
final_df['labels'].unique()

array([4, 1, 3, 2, 5])

In [53]:
final_df.head(5)  #USAR FINAL DF

,Title,Instructions,Image_Name,Cleaned_Ingredients,words,labels,Topic_Perc_Contrib,Keywords
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","[broth, squash, wine, sage, salt, bread, flour...",4,0.8326,"salt, pepper, butter, onion, garlic, vinegar, ..."
1,Crispy Salt and Pepper Potatoes,Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (...","[salt, parsley, thyme, pepper, potato]",1,0.8634,"salt, garlic, pepper, clove, onion, butter, to..."
2,Thanksgiving Mac and Cheese,Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ...","[macaroni, smoke, cream, milk, salt, cheddar, ...",3,0.5655,"salt, sugar, butter, flour, pepper, powder, cr..."
3,Italian Sausage and Bread Stuffing,Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in...","[broth, cream, rib, egg, onion, parsley, butte...",1,0.9453,"salt, garlic, pepper, clove, onion, butter, to..."
4,Newton's Law,Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho...","[bourbon, cinnamon, water, lemon, garnish, but...",2,0.8983,"sugar, butter, salt, water, flour, cream, clov..."


In [54]:
final_df[final_df['Image_Name'] == 'the-ultimate-hamburger-232191']

,Title,Instructions,Image_Name,Cleaned_Ingredients,words,labels,Topic_Perc_Contrib,Keywords
12492,The Ultimate Hamburger,1. Set up the grill for direct grilling and pr...,the-ultimate-hamburger-232191,"['1 1/4 pounds ground chuck', '1 1/4 pounds gr...","[sirloin, iceberg, chuck, salt, bread, butter,...",4,0.9185,"salt, pepper, butter, onion, garlic, vinegar, ..."


In [55]:
final_df[final_df['Image_Name'] == 'yellow-layer-cake-with-chocolate-sour-cream-frosting-51118090']

,Title,Instructions,Image_Name,Cleaned_Ingredients,words,labels,Topic_Perc_Contrib,Keywords
6010,Yellow Layer Cake with Chocolate-Sour Cream Fr...,Arrange racks in upper and lower thirds of ove...,yellow-layer-cake-with-chocolate-sour-cream-fr...,"['Nonstick vegetable oil spray', '4 cups cake ...","[cream, buttermilk, egg, vanilla, salt, corn, ...",3,0.9459,"salt, sugar, butter, flour, pepper, powder, cr..."


In [56]:
final_df[final_df['Title'] == "Newton's Law"]

,Title,Instructions,Image_Name,Cleaned_Ingredients,words,labels,Topic_Perc_Contrib,Keywords
4,Newton's Law,Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho...","[bourbon, cinnamon, water, lemon, garnish, but...",2,0.8983,"sugar, butter, salt, water, flour, cream, clov..."


In [57]:
show = final_df[(final_df['Image_Name'] == 'the-ultimate-hamburger-232191') | (final_df['Image_Name'] == 'yellow-layer-cake-with-chocolate-sour-cream-frosting-51118090') | (final_df['Title'] == "Newton's Law")]

In [66]:
show.reset_index(drop=True, inplace=True)

In [69]:
show.to_csv('show.csv')

In [58]:
final_df.groupby('labels')['labels'].count()

labels
1    2564
2    2234
3    3422
4    3658
5    1599
Name: labels, dtype: int64

In [59]:
final_df['label_names'] = np.NaN 
final_df.iloc[final_df[final_df['labels'] == 1].index, 8] = 'a'  
final_df.iloc[final_df[final_df['labels'] == 2].index, 8] = 'b' 
final_df.iloc[final_df[final_df['labels'] == 3].index, 8] = 'c' 
final_df.iloc[final_df[final_df['labels'] == 4].index, 8] = 'd' 
final_df.iloc[final_df[final_df['labels'] == 5].index, 8] = 'e'

In [60]:
final_df.head(5)

,Title,Instructions,Image_Name,Cleaned_Ingredients,words,labels,Topic_Perc_Contrib,Keywords,label_names
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","[broth, squash, wine, sage, salt, bread, flour...",4,0.8326,"salt, pepper, butter, onion, garlic, vinegar, ...",d
1,Crispy Salt and Pepper Potatoes,Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (...","[salt, parsley, thyme, pepper, potato]",1,0.8634,"salt, garlic, pepper, clove, onion, butter, to...",a
2,Thanksgiving Mac and Cheese,Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ...","[macaroni, smoke, cream, milk, salt, cheddar, ...",3,0.5655,"salt, sugar, butter, flour, pepper, powder, cr...",c
3,Italian Sausage and Bread Stuffing,Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in...","[broth, cream, rib, egg, onion, parsley, butte...",1,0.9453,"salt, garlic, pepper, clove, onion, butter, to...",a
4,Newton's Law,Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho...","[bourbon, cinnamon, water, lemon, garnish, but...",2,0.8983,"sugar, butter, salt, water, flour, cream, clov...",b


In [61]:
def img_label(df, labels, img_names): 
    img_path = os.path.join(os.path.abspath(os.curdir), 'images') 
    os.chdir(img_path)  
    for label in df[labels].unique(): 
        os.mkdir(label)  
        it = df[df[labels] == label]
        for j in range(len(it)): 
            try:
                f_name = os.path.join(os.path.abspath(os.curdir), it.iloc[j, 2]+'.jpg')
                f_renamed = os.path.join(os.path.abspath(os.curdir), it.iloc[j, 8]+'_{}'.format(j)+'.jpg')
                os.rename(f_name, f_renamed)
                shutil.move(f_renamed, os.path.join(os.path.abspath(os.curdir), label)) 
            except: 
                continue

In [62]:
#img_label(final_df, 'label_names', 'Image_Name')

In [63]:
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.9706,"salt, garlic, pepper, clove, onion, butter, to...","[broth, parmesan, russet, wine, salt, beef, fl..."
1,1.0,0.9627,"sugar, butter, salt, water, flour, cream, clov...","[fennel, flavor, salt, spice, flour, powder, p..."
2,2.0,0.9627,"salt, sugar, butter, flour, pepper, powder, cr...","[buttermilk, salt, flour, paprika, skin, garli..."
3,3.0,0.9685,"salt, pepper, butter, onion, garlic, vinegar, ...","[fennel, wine, lobster, salt, mint, skin, bulb..."
4,4.0,0.9704,"sugar, garlic, salt, clove, lime, pepper, wate...","[daikon, salt, radish, cilantro, plum, palm, b..."


In [64]:
for i in range(len(sent_topics_sorteddf_mallet)): 
    print('topic {}:'.format(i+1), sent_topics_sorteddf_mallet.iloc[i, 2], '\n')

topic 1: salt, garlic, pepper, clove, onion, butter, tomato, water, parsley, potato 

topic 2: sugar, butter, salt, water, flour, cream, clove, lemon, garlic, cinnamon 

topic 3: salt, sugar, butter, flour, pepper, powder, cream, egg, milk, lemon 

topic 4: salt, pepper, butter, onion, garlic, vinegar, wine, lemon, sugar, tomato 

topic 5: sugar, garlic, salt, clove, lime, pepper, water, ginger, chile, onion 

